# BIG5 문장 embedding vector 변환

### Model, Tokenizer Load

In [103]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")
model = AutoModel.from_pretrained("sentence-transformers/stsb-xlm-r-multilingual")

In [104]:
import pandas as pd
f = pd.read_csv('./data/Korean_big5_Question.csv', skiprows=[1], encoding='cp949')

In [105]:
f

,No.,Question_KO,Question_EN,Facet,Unnamed: 4,is_verified
0,1,나는 다른 사람들에 대해 관심이 있다.,Am interested in people.,A,1,Y
1,2,나는 다른 사람들의 감정에 잘 공감한다.,Sympathize with others' feelings.,A,1,Y
2,3,나는 마음이 여린 편이다.,Have a soft heart.,A,1,Y
3,4,나는 주변 다른 사람들에게 내 시간을 잘 할애하는 편이다.,Take time out for others.,A,1,Y
4,5,나는 주변 다른 사람들의 감정을 잘 알아차린다.,Feel others' emotions.,A,1,Y
...,...,...,...,...,...,...
156,157,미술관에 가는 것을 즐기지 않는다.,Do not enjoy going to art museums.,O,-1,NaN
157,158,나는 시를 좋아하지 않는다.,Do not like poetry.,O,-1,NaN
158,159,사물에 있어서 더 깊은 의미를 찾는 일은 드물다고 생각한다.,Rarely look for a deeper meaning in things.,O,-1,NaN
159,160,너무 많은 세금이 예술가를 지원하는 데 쓰인다고 생각한다.,Believe that too much tax money goes to suppor...,O,-1,NaN


In [106]:
f.columns

Index(['No.', 'Question_KO', 'Question_EN', 'Facet', 'Unnamed: 4',
       'is_verified'],
      dtype='object')

In [107]:
big5_Q = list(f['Question_KO'])
encoded_input = tokenizer(big5_Q, padding=True, truncation=True, max_length=128, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
big5_embedding_output = mean_pooling(model_output, encoded_input['attention_mask'])

In [108]:
# 가장 유사한 문장을 구해 성향 파악
def personality_search(text):
    encoded_input = tokenizer([text], padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        
    text_embedding_output = mean_pooling(model_output, encoded_input['attention_mask'])
    
    mymax = 0
    myidx = 99999
    
    cos_sim = torch.nn.CosineSimilarity(dim=0)
    for i, val in enumerate(big5_embedding_output):
        tmp = cos_sim(text_embedding_output[0], val)
        if mymax < tmp:
            mymax = tmp
            myidx = i
#     print(big5_embedding_output[myidx])
    print(f'source : {text}, target : {f["Question_KO"][myidx]}, personality : {f["Facet"][myidx]}, flag : {f["Unnamed: 4"][myidx]}, similarity : {mymax:.4f}')
    return mymax

In [101]:
personality_search('오늘 저녁은 치킨인가')

source : 오늘 저녁은 치킨인가, target : 남에게서 물러나고 싶다, personality : E, flag : -1, similarity : 0.8625


tensor(0.8625)

In [102]:
personality_search('이건 어떻게 되는거야? 오늘 저녁 뭘 먹을까?')

source : 이건 어떻게 되는거야? 오늘 저녁 뭘 먹을까?, target : 남에게서 물러나고 싶다, personality : E, flag : -1, similarity : 0.7748


tensor(0.7748)

In [95]:
tokenizer.tokenize('남에게서 물러나고 싶다')

['▁남', '에게', '서', '▁물', '러', '나', '고', '▁싶다']

In [96]:
tokenizer.tokenize('치킨은 맛있다')

['▁치', '킨', '은', '▁맛있', '다']